In [174]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd


In [209]:
spark = (
    SparkSession.builder
                .appName("Stack Overflow Data Wrangling")
                .config("spark.jars","C:\\Users\\Z\\Documents\\blossom\\jars\\postgresql-42.2.8.jar") 
                .getOrCreate()
)

In [141]:
spark = SparkSession.builder.getOrCreate()
questions = spark.read.csv("C:\\Users\\Z\\Documents\\blossom\\Projects\\Project3\\questions.csv", header=True, inferSchema=True)
answers = spark.read.csv("C:\\Users\\Z\\Documents\\blossom\\Projects\\Project3\\answers.csv" , header=True, inferSchema=True)
users = spark.read.csv("C:\\Users\\Z\\Documents\\blossom\\Projects\\Project3\\users.csv", header=True, inferSchema=True)

In [145]:
questions = questions.withColumnRenamed('id', 'questions_id')
answers = answers.withColumnRenamed('id', 'answers_id')
users = users.withColumnRenamed('id', 'users_id')
questions = questions.withColumnRenamed('created_at', 'questions_created_at')
answers = answers.withColumnRenamed('created_at', 'answers_created_at')
users = users.withColumnRenamed('created_at', 'users_created_at')
questions = questions.withColumnRenamed( 'body','questions_body')
answers = answers.withColumnRenamed( 'body','answers_body')
questions = questions.withColumnRenamed('user_id', 'questions_user_id')
answers = answers.withColumnRenamed('user_id', 'answers_user_id')
answers = answers.withColumnRenamed('question_id', 'answers_question_id')
questions = questions.withColumnRenamed( 'score','questions_score')
answers = answers.withColumnRenamed( 'score','answers_score')
questions = questions.withColumnRenamed( 'comment_count','questions_comment_count')
answers = answers.withColumnRenamed( 'comment_count','answers_comment_count')

In [148]:
answers.columns


['answers_id',
 'answers_user_id',
 'answers_question_id',
 'answers_body',
 'answers_score',
 'answers_comment_count',
 'answers_created_at']

In [151]:
users.registerTempTable('new_users')
new_users = spark.sql("select * from new_users where location like '%India%' ")
new_users.columns

['users_id',
 'display_name',
 'reputation',
 'website_url',
 'location',
 'about_me',
 'views',
 'up_votes',
 'down_votes',
 'image_url',
 'users_created_at',
 'updated_at']

In [152]:
new_users_updated =new_users.select('users_id', 'display_name', 'reputation', 'website_url', 
                                    'location', 'about_me', 'views', 'up_votes', 
                                    'down_votes', 'image_url', 'users_created_at', 
                                    'updated_at', F.split(new_users['location'], ',')[0].alias('city'), 
                                   F.split(new_users['location'], ',')[2].alias('country'))

In [153]:
new_users_updated.select('city','country').show(100)

+-------------+------------+
|         city|     country|
+-------------+------------+
|    Bangalore|       India|
|    New Delhi|       India|
|    Gharaunda|       India|
|    New Delhi|        null|
|    Jalandhar|       India|
|        Surat|       India|
|   Darjeeling|       India|
|         Pune|       India|
|       Mumbai|       India|
|    Bangalore|       India|
|       Mumbai|       India|
|    Bangalore|       India|
|       Mumbai|       India|
|    Hyderabad|       India|
|       Indore|       India|
|        India|        null|
|    Bangalore|       India|
|  Naya Raipur|       India|
|    Bangalore|       India|
|      Chennai|       India|
|    Bangalore|       India|
|         Pune|       India|
|         Pune|       India|
|        Delhi|        null|
|    Bangalore|       India|
|       Mumbai|       India|
|      Kolkata|       India|
|    Bangalore|       India|
|      Chennai|       India|
|    Bangalore|       India|
|        Surat|       India|
|    Hyderabad

In [157]:
#new_users_updated = users.withColumnRenamed('user_id', 'new_user_id')
new_users_questions = new_users_updated.join(questions, new_users_updated.users_id == questions.questions_user_id)

In [159]:
new_users_questions.registerTempTable('new_users_questions_temp')
new_users_questions_temp = spark.sql("select * from new_users_questions_temp where view_count > 20")

In [162]:
#answers = answers.withColumnRenamed('user_id', 'new_user_id')
new_user_questions_answers = new_users_questions_temp.join(answers, new_users_questions_temp.users_id == answers.answers_user_id)

In [166]:
new_user_questions_answers.select('users_id',
 'display_name',
 'reputation',
 'website_url',
 'location',
 'about_me',
 'views',
 'up_votes',
 'down_votes',
 'image_url',
 'users_created_at',
 'updated_at',
 'city',
 'country',
 'questions_id',
 'questions_user_id',
 'title',
 'questions_body',
 'accepted_answer_id',
 'questions_score',
 'view_count',
 'questions_comment_count',
 'questions_created_at',
 'answers_id',
 'answers_user_id',
 'answers_question_id',
 'answers_body',
 'answers_score',
 'answers_comment_count',
 'answers_created_at').show(20)

+--------+------------+----------+-----------+--------+--------+-----+--------+----------+---------+----------------+----------+----+-------+------------+-----------------+-----+--------------+------------------+---------------+----------+-----------------------+--------------------+----------+---------------+-------------------+------------+-------------+---------------------+------------------+
|users_id|display_name|reputation|website_url|location|about_me|views|up_votes|down_votes|image_url|users_created_at|updated_at|city|country|questions_id|questions_user_id|title|questions_body|accepted_answer_id|questions_score|view_count|questions_comment_count|questions_created_at|answers_id|answers_user_id|answers_question_id|answers_body|answers_score|answers_comment_count|answers_created_at|
+--------+------------+----------+-----------+--------+--------+-----+--------+----------+---------+----------------+----------+----+-------+------------+-----------------+-----+--------------+-------

In [186]:


min_time = new_user_questions_answers['updated_at'].dt.time

In [213]:
new_user_questions_answers.dtypes

[('users_id', 'string'),
 ('display_name', 'string'),
 ('reputation', 'string'),
 ('website_url', 'string'),
 ('location', 'string'),
 ('about_me', 'string'),
 ('views', 'string'),
 ('up_votes', 'string'),
 ('down_votes', 'string'),
 ('image_url', 'string'),
 ('users_created_at', 'string'),
 ('updated_at', 'string'),
 ('city', 'string'),
 ('country', 'string'),
 ('questions_id', 'string'),
 ('questions_user_id', 'string'),
 ('title', 'string'),
 ('questions_body', 'string'),
 ('accepted_answer_id', 'string'),
 ('questions_score', 'string'),
 ('view_count', 'string'),
 ('questions_comment_count', 'string'),
 ('questions_created_at', 'string'),
 ('answers_id', 'string'),
 ('answers_user_id', 'string'),
 ('answers_question_id', 'string'),
 ('answers_body', 'string'),
 ('answers_score', 'string'),
 ('answers_comment_count', 'string'),
 ('answers_created_at', 'string')]

In [211]:
new_user_questions_answers.write.format("jdbc").options(
    url='jdbc:postgresql://localhost:5432/postgres',
    driver='org.postgresql.Driver',
    user='postgres',
    password='postgres',
    dbtable='stackoverflow_filtered.results'
).save(mode='append')


Py4JJavaError: An error occurred while calling o2455.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 115.0 failed 1 times, most recent failure: Lost task 3.0 in stage 115.0 (TID 496, localhost, executor driver): java.io.FileNotFoundException: C:\Users\Z\AppData\Local\Temp\blockmgr-ec6f321a-8ebf-4910-810e-aeca40f8dbb9\30\temp_shuffle_b24bdcaa-2758-4135-854a-27ad4bbba28f (The system cannot find the path specified)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(Unknown Source)
	at java.io.FileOutputStream.<init>(Unknown Source)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:103)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:116)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:237)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java:151)

In [ ]:
#Difference between views and materialized view

""" The basic difference between View and Materialized View is that Views are not stored physically on the disk. 
On the other hands, Materialized Views are stored on the disc.

View can be defined as a virtual table created as a result of the query expression. However, Materialized View is a physical copy, picture or snapshot of the base table.

A view is always updated as the query creating View executes each time the View is used. On the other hands, Materialized View is updated manually or by applying triggers to it.

Materialized View responds faster than View as the Materialized View is precomputed.

Materialized View utilizes the memory space as it stored on the disk whereas, the View is just a display hence it do not require memory space.""""

In [ ]:
# pascalbros is the user I will award since he has the highest answer score of 277